# STORM with local documents

This notebook shows how to run [STORM](https://github.com/stanford-oval/storm) on local documents using US FEMA disaster preparation documentation.

The notebook ...

1. Downloads some FEMA disaster-related documents
2. Parses and chunks 
3. Embeds with local "BAAI/bge-m3"
4. Creates a local filesystem Qdrant vector store
5. Runs STORM using this store

# Setup

1. See [README](./README) to set up a conda environment and `.env` file


In [70]:
import os
import openai
from dotenv import load_dotenv
import os
import pandas as pd
import requests
from uuid import uuid4
import json
import sys

from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores.chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

from knowledge_storm import (
    STORMWikiRunnerArguments,
    STORMWikiRunner,
    STORMWikiLMConfigs,
)
from knowledge_storm.rm import VectorRM
from knowledge_storm.lm import OpenAIModel, AzureOpenAIModel
from knowledge_storm.utils import load_api_key, QdrantVectorStoreManager
from knowledge_storm.lm import OllamaClient

from langchain_openai import ChatOpenAI

from dspy import Example

pd.set_option("display.max_colwidth", None)

# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI API client
openai.api_key = os.getenv("OPENAI_API_KEY")

DATA_DIR = "./data"
DB_DIR = f"{DATA_DIR}/db"
PDF_DIR = f"{DATA_DIR}/pdfs"
STORM_OUTPUT_DIR = f"{DATA_DIR}/storm_output"
DB_COLLECTION_NAME = "fema_docs_demo"
EMBEDDING_MODEL = "BAAI/bge-m3"
TEXT_CHUNK_SUMMARY_MODEL = "gpt-4o-mini"
STORM_OUTPUT_MARKDOWN_ARTICLE = "storm_gen_article_polished.md"

for dir in [DATA_DIR, PDF_DIR, DB_DIR, STORM_OUTPUT_DIR]:
    os.makedirs(dir, exist_ok=True)

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDING_MODEL, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Used for generating text chunk titles and descriptions
llm = ChatOpenAI(
    model=TEXT_CHUNK_SUMMARY_MODEL,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

sentence_transformers.SentenceTransformer : INFO     : Load pretrained SentenceTransformer: BAAI/bge-m3


In [2]:
vectors = embeddings.embed_query("Bagels are the best!")
num_vectors = len(vectors)

print(f"Number of vectors: {num_vectors}")

Number of vectors: 1024


# Analysis

## Indexing FEMA Disaster preparedness documents

### Get FEMA PDF documents


In [3]:
df = pd.read_csv(f"{DATA_DIR}/fema_docs.csv")
display(df)

,Source,URL,Extra instructions,Document
0,FEMA,https://www.fema.gov/emergency-managers/risk-management/hazard-mitigation-planning/risk-reduction-activities,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-10/fema_scenario_1-active_shooter-01102020.pdf
1,FEMA,https://www.fema.gov/emergency-managers/risk-management/hazard-mitigation-planning/risk-reduction-activities,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-10/fema_scenario_1_active_shooter_TTX_answer_key-01102020.pdf
2,FEMA,https://www.fema.gov/emergency-managers/risk-management/hazard-mitigation-planning/risk-reduction-activities,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-property_coastal-erosion.pdf
3,FEMA,https://www.fema.gov/emergency-managers/risk-management/hazard-mitigation-planning/risk-reduction-activities,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-property_earthquakes.pdf
4,FEMA,https://www.fema.gov/emergency-managers/risk-management/hazard-mitigation-planning/risk-reduction-activities,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-home_flooding.pdf
5,FEMA,https://www.fema.gov/emergency-managers/risk-management/hazard-mitigation-planning/risk-reduction-activities,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-property_severe-wind.pdf
6,FEMA,https://www.fema.gov/emergency-managers/risk-management/hazard-mitigation-planning/risk-reduction-activities,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/documents/fema_protect-your-property-storm-surge.pdf
7,FEMA,https://www.fema.gov/emergency-managers/risk-management/hazard-mitigation-planning/risk-reduction-activities,"Selected ""Protect my home from natural hazards""",https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-property_wildfire.pdf
8,FEMA,https://www.fema.gov/emergency-managers/individuals-communities/what-would-you-do-scenarios,NaN,https://www.fema.gov/sites/default/files/2020-10/fema_scenario_2_tornado-01102020.pdf
9,FEMA,https://www.fema.gov/emergency-managers/individuals-communities/what-would-you-do-scenarios,NaN,https://www.fema.gov/sites/default/files/2020-10/fema_scenario_2-tornado_TTX_answer_key-01102020.pdf


### Build Vector Database

First we will build a FEMA RAG chain for asnwering questions about preparing for disasters, using FEMA PDFs.


In [6]:
for doc_url in df["Document"]:
    print(f"Downloading {doc_url}")
    response = requests.get(doc_url)
    with open(f"{PDF_DIR}/{doc_url.split('/')[-1]}", "wb") as f:
        f.write(response.content)

### Index documents

We will use a very simple parser and chunking methodology to ingest documents for this demo.


### Parse PDFs


In [7]:
def parse_pdfs():
    """
    Parses all PDF files in the specified directory and loads their content.

    This function iterates through all files in the directory specified by PDF_DIR,
    checks if they have a .pdf extension, and loads their content using PyPDFLoader.
    The loaded content from each PDF is appended to a list which is then returned.

    Returns:
        list: A list containing the content of all loaded PDF documents.
    """
    docs = []
    pdfs = os.listdir(PDF_DIR)
    print(f"We have {len(pdfs)} pdfs")
    for pdf_file in pdfs:
        if not pdf_file.endswith(".pdf"):
            continue
        print(f"Loading PDF: {pdf_file}")
        file_path = f"{PDF_DIR}/{pdf_file}"
        loader = PyPDFLoader(file_path)
        docs = docs + loader.load()
        print(f"Loaded {len(docs)} documents")

    return docs


docs = parse_pdfs()
print(f"Created {len(docs)} pages from {len(df)} documents")

We have 33 pdfs
Loading PDF: fema_scenario_10_power_outage_answer_key_01102020.pdf
Loaded 2 documents
Loading PDF: fema_scenario_7-shelter_in_place_TTX_answer_key_01102020.pdf
Loaded 5 documents
Loading PDF: ready_12-ways-to-prepare_postcard.pdf
Loaded 7 documents
Loading PDF: fema_safeguard-critical-documents-and-valuables.pdf
Loaded 10 documents
Loading PDF: ready_document-and-insure-your-property.pdf
Loaded 16 documents
Loading PDF: fema_scenario_1-active_shooter-01102020.pdf
Loaded 18 documents
Loading PDF: fema_protect-your-property_wildfire.pdf
Loaded 26 documents
Loading PDF: fema_scenario_4-hurricane-01102020.pdf
Loaded 27 documents
Loading PDF: fema_scenario_10_power_outage_01102020.pdf
Loaded 28 documents
Loading PDF: fema_scenario_4_hurricane_flood_TTX_answer_key-01102020.pdf
Loaded 30 documents
Loading PDF: fema_scenario_11_winter_storm_01102020.pdf
Loaded 32 documents
Loading PDF: fema_protect-your-property_severe-wind.pdf
Loaded 44 documents
Loading PDF: fema_protect-your

### Chunk texts


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks")

Created 251 chunks


### Enrich metadata

STORM requires some extra metadata fields. For the demo we'll use an LLM to generate these, but most ingestion pipelines will have metadata set.


In [8]:
def summarize_text(text, prompt):
    """
    Generate a summary of some text based on the user's prompt

    Args:

    text (str) - the text to analyze
    prompt (str) - prompt instruction on how to summarize the text, eg 'generate a title'

    Returns:

    summary (text) - LLM-generated summary

    """
    messages = [
        (
            "system",
            "You are an assistant that gives very brief single sentence description of text.",
        ),
        ("human", f"{prompt} :: \n\n {text}"),
    ]
    ai_msg = llm.invoke(messages)
    summary = ai_msg.content
    return summary


def enrich_metadata(docs):
    """
    Uses an LLM to populate 'title' and 'description' for text chunks

    Args:

    docs (list) - list of LangChain documents

    Returns:

    docs (list) - list of LangChain documents with metadata fields populated

    """
    new_docs = []
    for doc in docs:

        # pdf name is last part of doc.metadata['source']
        pdf_name = doc.metadata["source"].split("/")[-1]

        # Find row in df where pdf_name is in URL
        row = df[df["Document"].str.contains(pdf_name)]
        page = doc.metadata["page"] + 1
        url = f"{row['Document'].values[0]}?id={str(uuid4())}#page={page}"

        # We'll use an LLM to generate a summary and title of the text, used by STORM
        # This is just for the demo, proper application would have better metadata
        summary = summarize_text(doc.page_content, prompt="Please describe this text:")
        title = summarize_text(
            doc.page_content, prompt="Please generate a 5 word title for this text:"
        )

        doc.metadata["description"] = summary
        doc.metadata["title"] = title
        doc.metadata["url"] = url
        doc.metadata["content"] = doc.page_content

        # print(json.dumps(doc.metadata, indent=2))
        new_docs.append(doc)

    print(f"There are {len(docs)} docs")

    return new_docs


docs = enrich_metadata(docs)
chunks = enrich_metadata(chunks)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


There are 99 docs
There are 251 docs


### Build vector stores

We will build two vector stores, one using PDF pages as documents, the second with finer chunks.


In [10]:
def build_vector_store(doc_type, docs):
    """
    Givena  list of LangChain docs, will embed and create a file-system Qdrant vector database.
    The folder includes doc_type in its name to avoid overwriting.

    Args:

    doc_type (str) - String to indicate level of document split, eg 'pages',
                     'chunks'. Used to name the database save folder
    docs (list) - List of langchain documents to embed and store in vector database

    Returns:

    Nothing returned by function, but db saved to f"{DB_DIR}_{doc_type}".

    """

    print(f"There are {len(docs)} docs")

    save_dir = f"{DB_DIR}_{doc_type}"

    # If save_dir exists remove it
    if os.path.exists(save_dir):
        print(f"Removing existing directory {save_dir}")
        os.system(f"rm -rf {save_dir}")

    print(f"Saving vectors to directory {save_dir}")

    client = QdrantClient(path=save_dir)

    client.create_collection(
        collection_name=DB_COLLECTION_NAME,
        vectors_config=VectorParams(size=num_vectors, distance=Distance.COSINE),
    )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=DB_COLLECTION_NAME,
        embedding=embeddings,
    )

    uuids = [str(uuid4()) for _ in range(len(docs))]

    vector_store.add_documents(documents=docs, ids=uuids)


build_vector_store("pages", docs)
build_vector_store("chunks", chunks)

There are 99 docs
Removing existing directory ./data/db_pages
Saving vectors to directory ./data/db_pages
There are 251 docs
Removing existing directory ./data/db_chunks
Saving vectors to directory ./data/db_chunks


### Base retriever test

Let's do a quick check, also read vectors from disk.


In [11]:
def test_retriever(doc_type, query, top_k=15):

    save_dir = f"{DB_DIR}_{doc_type}"

    # Remove DB_DIR/.lock
    if os.path.exists(f"{save_dir}/.lock"):
        os.remove(f"{save_dir}/.lock")

    print("Loading vector_store from disk")
    client = QdrantClient(path=save_dir)
    vector_store = QdrantVectorStore(
        client=client,
        collection_name=DB_COLLECTION_NAME,
        embedding=embeddings,
    )

    print(f"*****> {doc_type}")
    print(f"         Query: {query}\n\n")

    retriever = vector_store.as_retriever(search_kwargs={"k": top_k})

    results = retriever.invoke(query)
    for doc in results[0:3]:
        print("-------")
        print(json.dumps(doc.metadata))
        print(doc.page_content)


query = "How should I prepare my home for hurricanes?"
test_retriever("pages", query, top_k=2)
test_retriever("chunks", query, top_k=2)

Loading vector_store from disk
*****> pages
         Query: How should I prepare my home for hurricanes?


-------
{"source": "./data/pdfs/fema_protect-your-property_severe-wind.pdf", "page": 9, "description": "The text provides links to FEMA resources for protecting properties from wind and hurricane damage.", "title": "\"Protecting Property from Wind Hazards\"", "url": "https://www.fema.gov/sites/default/files/2020-11/fema_protect-your-property_severe-wind.pdf?id=3c6341c3-d1b9-4ebf-bbe1-794119f9dad0#page=10", "content": "FEMA_Wind_Hazard_Brochure_MH_V4.indd   10FEMA_Wind_Hazard_Brochure_MH_V4.indd   10 1/13/20   4:30 PM1/13/20   4:30 PM  \n \n  \n \n \n \n  \n \n \n \n \n ADDITIONAL RESOURCES \nFEMA, PROTECT YOUR PROPERTY \nLearn how to protect your home or business from natural disasters \nfema.gov/protect-your-property \nFEMA, AGAINST THE WIND: PROTECTING YOUR \nHOME FROM HURRICANE AND WIND DAMAGE View illustrations that explain wind mitigation techniques \nfema.gov/media-library/a

## Run STORM Using our local document vectors

From the STORM [examples](https://github.com/stanford-oval/storm/blob/main/examples/storm_examples/README.md) ...


In [67]:
def set_instructions(runner):
    """
    Adjusts templates and personas for the STORM AI Research algorithm.

    Args:

    runner - STORM runner object

    Returns:

    runner - STORM runner object with extra prompting

    """

    # Open LMs are generally weaker in following output format.
    # One way for mitigation is to add one-shot example to the prompt to exemplify the desired output format.
    # For example, we can add the following examples to the two prompts used in StormPersonaGenerator.
    # Note that the example should be an object of dspy.Example with fields matching the InputField
    # and OutputField in the prompt (i.e., dspy.Signature).
    find_related_topic_example = Example(
        topic="Knowledge Curation",
        related_topics="https://en.wikipedia.org/wiki/Knowledge_management\n"
        "https://en.wikipedia.org/wiki/Information_science\n"
        "https://en.wikipedia.org/wiki/Library_science\n",
    )
    gen_persona_example = Example(
        topic="Knowledge Curation",
        examples="Title: Knowledge management\n"
        "Table of Contents: History\nResearch\n  Dimensions\n  Strategies\n  Motivations\nKM technologies"
        "\nKnowledge barriers\nKnowledge retention\nKnowledge audit\nKnowledge protection\n"
        "  Knowledge protection methods\n    Formal methods\n    Informal methods\n"
        "  Balancing knowledge protection and knowledge sharing\n  Knowledge protection risks",
        personas="1. Historian of Knowledge Systems: This editor will focus on the history and evolution of knowledge curation. They will provide context on how knowledge curation has changed over time and its impact on modern practices.\n"
        "2. Information Science Professional: With insights from 'Information science', this editor will explore the foundational theories, definitions, and philosophy that underpin knowledge curation\n"
        "3. Digital Librarian: This editor will delve into the specifics of how digital libraries operate, including software, metadata, digital preservation.\n"
        "4. Technical expert: This editor will focus on the technical aspects of knowledge curation, such as common features of content management systems.\n"
        "5. Museum Curator: The museum curator will contribute expertise on the curation of physical items and the transition of these practices into the digital realm.",
    )
    runner.storm_knowledge_curation_module.persona_generator.create_writer_with_persona.find_related_topic.demos = [
        find_related_topic_example
    ]
    runner.storm_knowledge_curation_module.persona_generator.create_writer_with_persona.gen_persona.demos = [
        gen_persona_example
    ]

    # A trade-off of adding one-shot example is that it will increase the input length of the prompt. Also, some
    # examples may be very long (e.g., an example for writing a section based on the given information), which may
    # confuse the model. For these cases, you can create a pseudo-example that is short and easy to understand to steer
    # the model's output format.
    # For example, we can add the following pseudo-examples to the prompt used in WritePageOutlineFromConv and
    # ConvToSection.
    write_page_outline_example = Example(
        topic="Example Topic",
        conv="Wikipedia Writer: ...\nExpert: ...\nWikipedia Writer: ...\nExpert: ...",
        old_outline="# Section 1\n## Subsection 1\n## Subsection 2\n"
        "# Section 2\n## Subsection 1\n## Subsection 2\n"
        "# Section 3",
        outline="# New Section 1\n## New Subsection 1\n## New Subsection 2\n"
        "# New Section 2\n"
        "# New Section 3\n## New Subsection 1\n## New Subsection 2\n## New Subsection 3",
    )
    runner.storm_outline_generation_module.write_outline.write_page_outline.demos = [
        write_page_outline_example
    ]
    write_section_example = Example(
        info="[1]\nInformation in document 1\n[2]\nInformation in document 2\n[3]\nInformation in document 3",
        topic="Example Topic",
        section="Example Section",
        output="# Example Topic\n## Subsection 1\n"
        "This is an example sentence [1]. This is another example sentence [2][3].\n"
        "## Subsection 2\nThis is one more example sentence [1].",
    )
    runner.storm_article_generation.section_gen.write_section.demos = [
        write_section_example
    ]

    return runner

def latest_dir(parent_folder):
    """
    Find the most recent folder (by modified date) in the specified parent folder.

    Args:
        parent_folder (str): The path to the parent folder where the search for the most recent folder will be conducted. Defaults to f"{DATA_DIR}/storm_output".

    Returns:
        str: The path to the most recently modified folder within the parent folder.
    """
    # Find most recent folder (by modified date) in DATA_DIR/storm_data
    # TODO, find out how exactly storm passes back its output directory to avoid this hack
    folders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]
    folder = max(folders, key=os.path.getmtime)

    return folder


def generate_footnotes(folder):
    """
    Generates footnotes from a JSON file containing URL information.

    Args:
        folder (str): The directory path where the 'url_to_info.json' file is located.

    Returns:
        str: A formatted string containing footnotes with URLs and their corresponding titles.
    """

    file = f"{folder}/url_to_info.json"

    with open(file) as f:
        data = json.load(f)

    refs = {}
    for rec in data["url_to_unified_index"]:
        val = data["url_to_unified_index"][rec]
        title = data["url_to_info"][rec]["title"].replace('"', "")
        refs[val] = f"- {val} [{title}]({rec})"

    keys = list(refs.keys())
    keys.sort()

    footer = ""
    for key in keys:
        footer += f"{refs[key]}\n"

    return footer, refs


def generate_markdown_article(output_dir):
    """
    Generates a markdown article by reading a text file, appending footnotes, 
    and saving the result as a markdown file.

    The function performs the following steps:
    1. Retrieves the latest directory using the `latest_dir` function.
    2. Generates footnotes for the article using the `generate_footnotes` function.
    3. Reads the content of a text file named 'storm_gen_article_polished.txt' 
       located in the latest directory.
    4. Appends the generated footnotes to the end of the article content.
    5. Writes the modified content to a new markdown file named 
       STORM_OUTPUT_MARKDOWN_ARTICLE in the same directory.

    Args:

    output_dir (str) - The directory where the STORM output is stored.


    """

    folder = latest_dir(output_dir)
    footnotes, refs = generate_footnotes(folder)

    with open(f"{folder}/storm_gen_article_polished.txt") as f:
        text = f.read()

    # Update text references like [10] to link to URLs
    for ref in refs:
        print(f"Ref: {ref}, Ref_num: {refs[ref]}")
        url = refs[ref].split("(")[1].split(")")[0]
        text = text.replace(f"[{ref}]", f"\[[{ref}]({url})\]")

    text += f"\n\n## References\n\n{footnotes}"

    with open(f"{folder}/{STORM_OUTPUT_MARKDOWN_ARTICLE}", "w") as f:
        f.write(text)

def run_storm(topic, model_type, db_dir):
    """
    This function runs the STORM AI Research algorithm using data
    in a QDrant local database.

    Args:

    topic (str) - The research topic to generate the article for
    model_type (str) - One of 'openai' and 'ollama' to control LLM used
    db_dir (str) - Directory where the QDrant vector database is
    
    """
    if model_type not in ["openai", "ollama"]:
        print("Unsupported model_type")
        sys.exit()

    # Clear lock so can be read
    if os.path.exists(f"{db_dir}/.lock"):
        print(f"Removing lock file {db_dir}/.lock")
        os.remove(f"{db_dir}/.lock")

    print(f"Loading Qdrant vector store from {db_dir}")

    engine_lm_configs = STORMWikiLMConfigs()

    if model_type == "openai":

        print("Using OpenAI models")

        # Initialize the language model configurations
        openai_kwargs = {
            "api_key": os.getenv("OPENAI_API_KEY"),
            "temperature": 1.0,
            "top_p": 0.9,
        }

        ModelClass = (
            OpenAIModel
            if os.getenv("OPENAI_API_TYPE") == "openai"
            else AzureOpenAIModel
        )
        # If you are using Azure service, make sure the model name matches your own deployed model name.
        # The default name here is only used for demonstration and may not match your case.
        gpt_35_model_name = (
            "gpt-4o-mini"
            if os.getenv("OPENAI_API_TYPE") == "openai"
            else "gpt-35-turbo"
        )
        gpt_4_model_name = "gpt-4o"
        if os.getenv("OPENAI_API_TYPE") == "azure":
            openai_kwargs["api_base"] = os.getenv("AZURE_API_BASE")
            openai_kwargs["api_version"] = os.getenv("AZURE_API_VERSION")

        # STORM is a LM system so different components can be powered by different models.
        # For a good balance between cost and quality, you can choose a cheaper/faster model for conv_simulator_lm
        # which is used to split queries, synthesize answers in the conversation. We recommend using stronger models
        # for outline_gen_lm which is responsible for organizing the collected information, and article_gen_lm
        # which is responsible for generating sections with citations.
        conv_simulator_lm = ModelClass(
            model=gpt_35_model_name, max_tokens=10000, **openai_kwargs
        )
        question_asker_lm = ModelClass(
            model=gpt_35_model_name, max_tokens=10000, **openai_kwargs
        )
        outline_gen_lm = ModelClass(
            model=gpt_4_model_name, max_tokens=10000, **openai_kwargs
        )
        article_gen_lm = ModelClass(
            model=gpt_4_model_name, max_tokens=10000, **openai_kwargs
        )
        article_polish_lm = ModelClass(
            model=gpt_4_model_name, max_tokens=10000, **openai_kwargs
        )

    elif model_type == "ollama":

        print("Using Ollama models")

        ollama_kwargs = {
            # "model": "llama3.2:3b",
            "model": "llama3.1:latest",
            # "model": "qwen2.5:14b",
            "port": "11434",
            "url": "http://localhost",
            "stop": (
                "\n\n---",
            ),  # dspy uses "\n\n---" to separate examples. Open models sometimes generate this.
        }

        conv_simulator_lm = OllamaClient(max_tokens=500, **ollama_kwargs)
        question_asker_lm = OllamaClient(max_tokens=500, **ollama_kwargs)
        outline_gen_lm = OllamaClient(max_tokens=400, **ollama_kwargs)
        article_gen_lm = OllamaClient(max_tokens=700, **ollama_kwargs)
        article_polish_lm = OllamaClient(max_tokens=4000, **ollama_kwargs)

    engine_lm_configs.set_conv_simulator_lm(conv_simulator_lm)
    engine_lm_configs.set_question_asker_lm(question_asker_lm)
    engine_lm_configs.set_outline_gen_lm(outline_gen_lm)
    engine_lm_configs.set_article_gen_lm(article_gen_lm)
    engine_lm_configs.set_article_polish_lm(article_polish_lm)

    max_conv_turn = 4
    max_perspective = 3
    search_top_k = 10
    max_thread_num = 1
    device = "cpu"
    vector_db_mode = "offline"

    do_research = True
    do_generate_outline = True
    do_generate_article = True
    do_polish_article = True

    # Initialize the engine arguments
    output_dir=f"{STORM_OUTPUT_DIR}/{db_dir.split('db_')[1]}"
    print(f"Output directory: {output_dir}")

    engine_args = STORMWikiRunnerArguments(
        output_dir=output_dir,
        max_conv_turn=max_conv_turn,
        max_perspective=max_perspective,
        search_top_k=search_top_k,
        max_thread_num=max_thread_num,
    )

    # Setup VectorRM to retrieve information from your own data
    rm = VectorRM(
        collection_name=DB_COLLECTION_NAME,
        embedding_model=EMBEDDING_MODEL,
        device=device,
        k=search_top_k,
    )

    # initialize the vector store, either online (store the db on Qdrant server) or offline (store the db locally):
    if vector_db_mode == "offline":
        rm.init_offline_vector_db(vector_store_path=db_dir)

    # Initialize the STORM Wiki Runner
    runner = STORMWikiRunner(engine_args, engine_lm_configs, rm)

    # Set instructions for the STORM AI Research algorithm
    runner = set_instructions(runner)

    # run the pipeline
    runner.run(
        topic=topic,
        do_research=do_research,
        do_generate_outline=do_generate_outline,
        do_generate_article=do_generate_article,
        do_polish_article=do_polish_article,
    )
    runner.post_run()
    runner.summary()

    generate_markdown_article(output_dir)

In [68]:
topic = "Compare the financial impact of different types of disasters and how those impact communities"

for doc_type in ["pages", "chunks"]:
    db_dir = f"{DB_DIR}_{doc_type}"
    run_storm(topic=topic, model_type="openai", db_dir=db_dir)

sentence_transformers.SentenceTransformer : INFO     : Load pretrained SentenceTransformer: BAAI/bge-m3


Removing lock file ./data/db_pages/.lock
Loading Qdrant vector store from ./data/db_pages
Using OpenAI models
Output directory: ./data/storm_output/pages
Collection fema_docs_demo exists. Loading the collection...


knowledge_storm.interface : INFO     : run_knowledge_curation_module executed in 8.1188 seconds
knowledge_storm.interface : INFO     : run_outline_generation_module executed in 0.0041 seconds
sentence_transformers.SentenceTransformer : INFO     : Use pytorch device_name: mps
sentence_transformers.SentenceTransformer : INFO     : Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2
knowledge_storm.interface : INFO     : run_article_generation_module executed in 3.4558 seconds
knowledge_storm.interface : INFO     : run_article_polishing_module executed in 0.0018 seconds
sentence_transformers.SentenceTransformer : INFO     : Load pretrained SentenceTransformer: BAAI/bge-m3


***** Execution time *****
run_knowledge_curation_module: 8.1188 seconds
run_outline_generation_module: 0.0041 seconds
run_article_generation_module: 3.4558 seconds
run_article_polishing_module: 0.0018 seconds
***** Token usage of language models: *****
run_knowledge_curation_module
    gpt-4o-mini: {'prompt_tokens': 46235, 'completion_tokens': 10748}
    gpt-4o: {'prompt_tokens': 0, 'completion_tokens': 0}
run_outline_generation_module
    gpt-4o-mini: {'prompt_tokens': 0, 'completion_tokens': 0}
    gpt-4o: {'prompt_tokens': 6783, 'completion_tokens': 595}
run_article_generation_module
    gpt-4o-mini: {'prompt_tokens': 0, 'completion_tokens': 0}
    gpt-4o: {'prompt_tokens': 14167, 'completion_tokens': 2342}
run_article_polishing_module
    gpt-4o-mini: {'prompt_tokens': 0, 'completion_tokens': 0}
    gpt-4o: {'prompt_tokens': 2459, 'completion_tokens': 474}
***** Number of queries of retrieval models: *****
run_knowledge_curation_module: {'VectorRM': 48}
run_outline_generation_modu

knowledge_storm.interface : INFO     : run_knowledge_curation_module executed in 7.8170 seconds
knowledge_storm.interface : INFO     : run_outline_generation_module executed in 0.0037 seconds
sentence_transformers.SentenceTransformer : INFO     : Use pytorch device_name: mps
sentence_transformers.SentenceTransformer : INFO     : Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2
knowledge_storm.interface : INFO     : run_article_generation_module executed in 2.9605 seconds
knowledge_storm.interface : INFO     : run_article_polishing_module executed in 0.0026 seconds


***** Execution time *****
run_knowledge_curation_module: 7.8170 seconds
run_outline_generation_module: 0.0037 seconds
run_article_generation_module: 2.9605 seconds
run_article_polishing_module: 0.0026 seconds
***** Token usage of language models: *****
run_knowledge_curation_module
    gpt-4o-mini: {'prompt_tokens': 46564, 'completion_tokens': 10899}
    gpt-4o: {'prompt_tokens': 0, 'completion_tokens': 0}
run_outline_generation_module
    gpt-4o-mini: {'prompt_tokens': 0, 'completion_tokens': 0}
    gpt-4o: {'prompt_tokens': 6731, 'completion_tokens': 599}
run_article_generation_module
    gpt-4o-mini: {'prompt_tokens': 0, 'completion_tokens': 0}
    gpt-4o: {'prompt_tokens': 14119, 'completion_tokens': 2743}
run_article_polishing_module
    gpt-4o-mini: {'prompt_tokens': 0, 'completion_tokens': 0}
    gpt-4o: {'prompt_tokens': 2732, 'completion_tokens': 423}
***** Number of queries of retrieval models: *****
run_knowledge_curation_module: {'VectorRM': 48}
run_outline_generation_modu